In [1]:
import pandas as pd
import numpy as np

In [2]:
%ls

Freq2011.zip     freqrnc_readme.txt              selected words.txt
freqrnc2011.csv  Generate mnemonic system.ipynb  Untitled 1.xlsx


In [3]:
words = pd.read_csv('freqrnc2011.csv', sep='\t')
words.head(5)

,Lemma,PoS,Freq(ipm),R,D,Doc
0,а,conj,8198.0,100,97,32332
1,а,intj,19.8,99,90,757
2,а,part,6.1,59,79,128
3,а,s,2.7,59,85,160
4,аа,intj,1.5,47,80,68


In [5]:
print(words['PoS'].unique())

['conj' 'intj' 'part' 's' 's.PROP' 'a' 'v' 'adv' 'spro' 'pr' 'num' 'apro'
 'advpro' 'anum']


In [4]:
words = words[words['PoS']=='s']
words.drop(['PoS', 'R', 'D', 'Doc'], axis=1, inplace=True)
words = words.sort_values(by='Freq(ipm)', ascending=False)
words.head()

,Lemma,Freq(ipm)
8015,год,3727.5
49992,человек,2723.0
5955,время,2015.7
9304,дело,1412.1
11456,жизнь,1389.8


In [5]:
print(words.shape)

(22050, 2)


In [6]:
soglasniye_all = 'бвгджзклмнпрстфхцчшщ'
soglasniye_easy = 'нтрсзжлвкпмдгц'
soglasniye_difficult = 'фхчшщ'

In [7]:
words['allchars'] = 0
words['easychars'] = 0
words['difficultchars'] = 0
words['chars'] = ''
words.head(1)

,Lemma,Freq(ipm),allchars,easychars,difficultchars,chars
8015,год,3727.5,0,0,0,


In [8]:
def numberofchars(word, chars):
    count = 0
    for char in word:
        if char in chars:
            count += 1
    return count

def charsinword(word, chars):
    result = ''
    for char in word:
        if char in chars:
            result += char
    return result

for i in range(0, words.shape[0]):
    word = words.iloc[i, 0]
    words.iloc[i, 2] = numberofchars(word, soglasniye_all)
    words.iloc[i, 3] = numberofchars(word, soglasniye_easy)
    words.iloc[i, 4] = numberofchars(word, soglasniye_difficult)
    words.iloc[i, 5] = charsinword(word, soglasniye_all)
words.head()

,Lemma,Freq(ipm),allchars,easychars,difficultchars,chars
8015,год,3727.5,2,2,0,гд
49992,человек,2723.0,4,3,1,члвк
5955,время,2015.7,3,3,0,врм
9304,дело,1412.1,2,2,0,дл
11456,жизнь,1389.8,3,3,0,жзн


In [9]:
cond01 = words['allchars'] < 4
cond02 = words['allchars'] > 1
cond03 = words['difficultchars'] == 0
easy_words = words[cond01 & cond02 & cond03]
for i in range(1, easy_words.shape[0] // 500):
    start = i*500
    end = start + 5
    print(start, [easy_words.iloc[x, 0] for x in range(start, end)])

500 ['гриб', 'полиция', 'цирк', 'благо', 'манера']
1000 ['рев', 'скука', 'насос', 'удаление', 'гусь']
1500 ['молот', 'гетто', 'блядь', 'серб', 'синод']
2000 ['неверие', 'скок', 'неволя', 'осада', 'отвал']
2500 ['бузина', 'восьмая', 'типаж', 'излом', 'задел']
3000 ['абрек', 'ларь', 'аврал', 'львица', 'юниор']
3500 ['рвань', 'булава', 'атолл', 'тейп', 'спуд']


In [10]:
good_words = easy_words.iloc[0:2500, :]
print(good_words.shape)

(2500, 6)


In [11]:
good_words['allchars'].value_counts()    

3    1793
2     707
Name: allchars, dtype: int64

In [15]:
def getallcombos(mapping, ns):
    combos = []
    if len(ns) == 1:
        for char in mapping[ns[0]]:
            combos.append(char)
        return combos
    elif len(ns) > 1:
        next_combos = getallcombos(mapping, ns[1:])
        for next_combo in next_combos:
            for char in mapping[ns[0]]:
                combos.append(char+next_combo)
    return combos

def testmapping(mapping, words_for_test):
    result = []
    for number in range(10, 100):
        number_str = str(number)
        combos = getallcombos(mapping, number_str)
        word_count = 0
        for combo in combos:
            cond01 = words_for_test['chars']==combo
            word_list = words_for_test[cond01]['Lemma']
            word_count += len(word_list.tolist())
        result.append(word_count)
    return result

def testresult(words_count):
    #print('len:', len(words_count))
    #print('min:', np.min(words_count))
    #print('mean:', np.mean(words_count))
    #print('median:', np.median(words_count))
    #print('max:', np.max(words_count))
    print('0-5 words:', [ len([ x for x in words_count if x<y ]) for y in range(1,6) ] )
    #print('all of it:')
    #print(words_count)

In [18]:
test_mapping01 = {
    '1': 'жг',
    '2': 'дт',
    '3': 'хк',
    '4': 'чщ',
    '5': 'бп',
    '6': 'лш',
    '7': 'сз',
    '8': 'вф',
    '9': 'цр',
    '0': 'мн'
}
testresult(testmapping(test_mapping01, words))
testresult(testmapping(test_mapping01, good_words))

0-5 words: [0, 4, 7, 11, 15]
0-5 words: [22, 23, 28, 34, 38]


In [19]:
test_mapping02 = {
    '1': 'р',
    '2': 'д',
    '3': 'т',
    '4': 'ч',
    '5': 'п',
    '6': 'ш',
    '7': 'с',
    '8': 'в',
    '9': 'м',
    '0': 'н'
}
testresult(testmapping(test_mapping02, words))
testresult(testmapping(test_mapping02, good_words))

0-5 words: [8, 18, 32, 40, 46]
0-5 words: [41, 47, 51, 61, 66]


In [20]:
naive_mapping01 = {
    '1': 'н',
    '2': 'т',
    '3': 'р',
    '4': 'сзж',
    '5': 'л',
    '6': 'в',
    '7': 'к',
    '8': 'п',
    '9': 'м',
    '0': 'дг'
}
naive_mapping02 = {
    '1': 'дг',
    '2': 'н',
    '3': 'м',
    '4': 'р',
    '5': 'п',
    '6': 'т',
    '7': 'сзж',
    '8': 'в',
    '9': 'к',
    '0': 'л'
}
testresult(testmapping(naive_mapping01, words))
testresult(testmapping(naive_mapping02, words))

0-5 words: [4, 8, 11, 15, 19]
0-5 words: [4, 9, 11, 14, 18]


К сожалению, "наивный" подход дает результат хуже, чем базовый вариант.
Может, это не так на простых словах?

In [29]:
testresult(testmapping(test_mapping01, good_words))
testresult(testmapping(naive_mapping01, good_words))
testresult(testmapping(naive_mapping02, good_words))

0-5 words: [22, 23, 28, 34, 38]
0-5 words: [8, 13, 24, 34, 40]
0-5 words: [8, 13, 22, 33, 39]


И действительно, среди "простых" слов наивный маппинг выглядит чуть лучше из-за меньшего количества нулей, при близких средних и медианах. К сожалению, всё равно много чисел с небольшим числом кодов.

Сделаем комбинацию всех букв с учетом их частоты во всем языке.

In [30]:
naive_mapping03 = {
    '1': 'т',
    '2': 'р',
    '3': 'с',
    '4': 'чгк',
    '5': 'вф',
    '6': 'пб',
    '7': 'мжшщ',
    '8': 'лх',
    '9': 'здц',
    '0': 'н'
}
testresult(testmapping(naive_mapping03, words))
testresult(testmapping(naive_mapping03, good_words))

0-5 words: [1, 1, 5, 5, 7]
0-5 words: [6, 9, 15, 18, 25]


Такой мэппинг лучше работает и с точки зрения всех слов, и на "простых" словах. Чтобы улучшить работу на простых, в частности, исключить нули, можно попробовать посчитать частоту в этих "простых" словах.

In [31]:
cond01 = good_words['allchars'] == 2
gw = good_words[cond01]['Lemma'].tolist()
cond01 = words['allchars'] == 2
aw = words[cond01]['Lemma'].tolist()
freqsgw = {}
freqsaw = {}
for char in soglasniye_all:
    freqgw = 0
    for word in gw:
        if char in word:
            freqgw += 1
    freqsgw[char] = freqgw / len(gw)
    freqaw = 0
    for word in aw:
        if char in word:
            freqaw += 1
    freqsaw[char] = freqaw / len(aw)
print(freqsgw)
print(freqsaw)

{'ч': 0.0, 'щ': 0.0, 'н': 0.15558698727015557, 'д': 0.13861386138613863, 'л': 0.21923620933521923, 'б': 0.10325318246110325, 'з': 0.10466760961810467, 'к': 0.17114568599717114, 'п': 0.13578500707213578, 'р': 0.22347949080622348, 'м': 0.13437057991513437, 'г': 0.08062234794908062, 'х': 0.0, 'в': 0.09476661951909476, 'т': 0.16407355021216408, 'ц': 0.031117397454031116, 'с': 0.15558698727015557, 'ф': 0.0, 'ш': 0.0, 'ж': 0.055162659123055166}
{'ч': 0.05662983425414365, 'щ': 0.021408839779005526, 'н': 0.1526243093922652, 'д': 0.12292817679558012, 'л': 0.18577348066298344, 'б': 0.09668508287292818, 'з': 0.0718232044198895, 'к': 0.1712707182320442, 'п': 0.11533149171270718, 'р': 0.19958563535911603, 'м': 0.12292817679558012, 'г': 0.07113259668508287, 'х': 0.04488950276243094, 'в': 0.07803867403314917, 'т': 0.16574585635359115, 'ц': 0.026243093922651933, 'с': 0.12914364640883977, 'ф': 0.04143646408839779, 'ш': 0.04903314917127072, 'ж': 0.04143646408839779}


In [32]:
naive_mapping04 = {
    '1': 'л',
    '2': 'р',
    '3': 'к',
    '4': 'н',
    '5': 'с',
    '6': 'т',
    '7': 'д',
    '8': 'мжг',
    '9': 'пб',
    '0': 'вз'
}
testresult(testmapping(naive_mapping04, words))
testresult(testmapping(naive_mapping04, good_words))

0-5 words: [1, 2, 5, 9, 16]
0-5 words: [2, 8, 21, 25, 32]


А что, если учесть позиции - первое или второе место?

In [21]:
cond01 = words['allchars'] == 2
gw = words[cond01]['chars'].tolist()
freqsgw_1 = {}
freqsgw_2 = {}
freqsgw_12 = {}
for char in soglasniye_all:
    freqgw_1 = 0
    freqgw_2 = 0
    freqgw_12 = 0
    for word in gw:
        if char in word[0]:
            freqgw_1 += 1
        if char in word[1]:
            freqgw_2 += 1
        if char+char==word:
            freqgw_12 += 1
    freqsgw_1[char] = freqgw_1 / len(gw)
    freqsgw_2[char] = freqgw_2 / len(gw)
    freqsgw_12[char] = freqgw_12 / len(gw)

In [22]:
for char in soglasniye_all:
    print(char, 
          round(freqsgw_1[char]*100, 2),
          round(freqsgw_2[char]*100, 2),
          round(freqsgw_12[char]*100, 2),
          sep='\t')

б	6.49	3.45	0.28
в	4.97	2.9	0.07
г	3.87	3.25	0.0
д	6.77	5.8	0.28
ж	2.0	2.21	0.07
з	4.28	2.97	0.07
к	7.87	9.74	0.48
л	7.94	10.77	0.14
м	7.18	5.46	0.35
н	4.7	10.98	0.41
п	8.15	3.8	0.41
р	8.84	11.12	0.0
с	8.15	5.04	0.28
т	6.35	10.64	0.41
ф	2.49	1.73	0.07
х	2.42	2.07	0.0
ц	0.69	2.0	0.07
ч	3.45	2.35	0.14
ш	2.56	2.42	0.07
щ	0.83	1.31	0.0


Учитываем частоту в первом и втором слогах.

In [33]:
naive_mapping05 = {
    '1': 'рч',
    '2': 'нв',
    '3': 'л',
    '4': 'пт',
    '5': 'кф',
    '6': 'бцх',
    '7': 'жзщ',
    '8': 'сш',
    '9': 'м',
    '0': 'дг'
}
testresult(testmapping(naive_mapping05, words))
testresult(testmapping(naive_mapping05, good_words))

0-5 words: [0, 0, 2, 4, 5]
0-5 words: [1, 5, 13, 22, 25]


То же, что и 05, но в более разумном порядке

In [34]:
naive_mapping06 = {
    '1': 'нв',
    '2': 'кф',
    '3': 'мш',
    '4': 'чр',
    '5': 'пт',
    '6': 'бцх',
    '7': 'с',
    '8': 'зжщ',
    '9': 'дг',
    '0': 'л'
}
testresult(testmapping(naive_mapping06, words))
testresult(testmapping(naive_mapping06, good_words))

0-5 words: [0, 0, 1, 3, 5]
0-5 words: [1, 4, 11, 22, 25]


Проверим:

In [27]:
for number in [1, 11, 12, 100, 101, 102]:
    number_str = str(number)
    combos = getallcombos(naive_mapping06, number_str)
    words_found = []
    for combo in combos:
        cond01 = words['chars'] == combo
        words_found += words[cond01]['Lemma'].tolist()
    print(number, words_found)

1 ['июнь', 'ион', 'иней', 'иена', 'юань', 'но', 'инь', 'эн', 'ню', 'уния', 'эон', 'вой', 'ива', 'явь', 'вэ', 'айва', 'авиа', 'выя']
11 ['няня', 'уныние', 'нона', 'неон', 'анион', 'нюни', 'война', 'вино', 'вина', 'воин', 'вена', 'вонь', 'веяние', 'овен', 'авеню', 'вьюн', 'эвен', 'овин', 'ваяние', 'новое', 'нива', 'новь', 'авва']
12 ['наука', 'ник', 'инок', 'янки', 'инк', 'век', 'веко', 'явка', 'вояка', 'вьюк', 'инфа', 'неф']
100 ['вилла']
101 ['нейлон', 'влияние', 'волна', 'явление', 'валун', 'веление', 'авиалиния', 'вольный', 'увалень', 'волан', 'вольная', 'налив', 'вылов']
102 ['нолик', 'волк', 'вилка', 'валик', 'волок', 'валек', 'войлок', 'валок', 'валка', 'велик']


Для проверки наличия слова в том или ином списке использовался код:

In [35]:
test_word = 'лох'
print(words[words['Lemma']==test_word])
print(easy_words[easy_words['Lemma']==test_word])
print(good_words[good_words['Lemma']==test_word])

      Lemma  Freq(ipm)  allchars  easychars  difficultchars chars
19470   лох        4.6         2          1               1    лх
Empty DataFrame
Columns: [Lemma, Freq(ipm), allchars, easychars, difficultchars, chars]
Index: []
Empty DataFrame
Columns: [Lemma, Freq(ipm), allchars, easychars, difficultchars, chars]
Index: []
